In [14]:
pip install catboost


  Using cached catboost-1.2.7-cp310-cp310-win_amd64.whl.metadata (1.2 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached plotly-6.0.0-py3-none-any.whl.metadata (5.6 kB)
Using cached catboost-1.2.7-cp310-cp310-win_amd64.whl (101.8 MB)
Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)
Using cached plotly-6.0.0-py3-none-any.whl (14.8 MB)
   ---------------------------------------- 0.0/305.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/305.2 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/305.2 kB 1.3 MB/s eta 0:00:01
   ------- ------------------------------- 61.4/305.2 kB 465.5 kB/s eta 0:00:01
   -------------- ----------------------- 112.6/305.2 kB 652.2 kB/s eta 0:00:01
   --------------- ---------------------- 122.9/305.2 kB 599.1 kB/s eta 0:00:01
   ------------------------- -

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

In [2]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_Rainfall = pd.read_csv('data/Rainfall.csv')

id_test = df_test['id']

## Признаки:
- id (Порядковый номер)
- day (День года)
- pressure (Давление)
- maxtemp (Максимальная температура)
- temparature (Средняя температура)
- mintemp (Минимальная температура)
- dewpoint (Точка росы. Это температура, при которой атмосфера насыщается водяным паром при охлаждении без изменения давления или содержания пара.)
- humidity (Влажность)
- cloud (Облачность)
- sunshine (Солнечный свет)
- winddirection (Направление ветра)
- windspeed (Скорость ветра)
- rainfall (Осадки) - Целевой признак


в df_test['winddirection'] одно nan значение (заменяю на 0, отсутствие ветра) попробывать другие варианты

в df_Rainfall['winddirection'] одно nan значение и название с пробелами

в df_Rainfall['windspeed'] одно nan значение

типы данных различаются в df_train и df_Rainfall

In [4]:
df_test['winddirection'] = df_test['winddirection'].fillna(0)

df_Rainfall.rename(columns={'         winddirection': 'winddirection'}, inplace=True)



In [19]:
df_Rainfall.dtypes

day                int64
pressure         float64
maxtemp          float64
temparature      float64
mintemp          float64
dewpoint         float64
humidity           int64
cloud              int64
rainfall          object
sunshine         float64
winddirection    float64
windspeed        float64
dtype: object

In [18]:
df_train.dtypes

day                int64
pressure         float64
maxtemp          float64
temparature      float64
mintemp          float64
dewpoint         float64
humidity         float64
cloud            float64
sunshine         float64
winddirection    float64
windspeed        float64
rainfall           int64
dtype: object

In [16]:
df_Rainfall['winddirection'].value_counts()

winddirection
20.0     47
70.0     39
40.0     33
50.0     32
220.0    27
60.0     24
80.0     23
230.0    20
30.0     17
10.0     15
200.0    10
190.0     9
240.0     8
100.0     7
90.0      7
180.0     6
210.0     6
130.0     5
110.0     4
170.0     3
120.0     3
160.0     3
270.0     3
300.0     2
150.0     2
280.0     2
250.0     2
290.0     2
140.0     2
260.0     1
350.0     1
Name: count, dtype: int64

In [5]:
df_train.drop(columns='id', inplace=True)
df_test.drop(columns='id', inplace=True)

In [6]:
X = df_train.drop(columns='rainfall')
y = df_train['rainfall']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
model = CatBoostClassifier(verbose = 0)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(roc_auc_score(y_test, y_pred))

0.7811438054845763
              precision    recall  f1-score   support

           0       0.80      0.62      0.70       119
           1       0.87      0.94      0.90       319

    accuracy                           0.85       438
   macro avg       0.83      0.78      0.80       438
weighted avg       0.85      0.85      0.85       438



In [8]:
pred = model.predict(df_test)

In [9]:
output = pd.DataFrame({'id': id_test, 'rainfall': pred})
output.to_csv('data/cat_model.csv', index=False)